In [3]:
from dotenv import load_dotenv
load_dotenv()

import streamlit as st
from PyPDF2 import PdfReader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_core.tools import create_retriever_tool
from langchain_classic.agents import AgentExecutor, create_tool_calling_agent
from langchain_community.embeddings import DashScopeEmbeddings
from langchain.chat_models import init_chat_model

import os

DeepSeek_API_KEY = os.getenv("DEEPSEEK_API_KEY")
DashScope_API_KEY = os.getenv("DASHSCOPE_API_KEY")

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

embedding_model = DashScopeEmbeddings(
    model="text-embedding-v4",
    dashscope_api_key=DashScope_API_KEY
)

def pdf_read(pdf_doc):
    if isinstance(pdf_doc, str):
        pdf_doc = [pdf_doc]
    text = []
    for pdf in pdf_doc:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text.append(page.extract_text())
    return "".join(text)

def get_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.split_text(text)
    return chunks

def vector_store(text_chunks):
    vs = FAISS.from_texts(text_chunks, embedding=embedding_model)
    vs.save_local("faiss_db")

def get_conversational_chain(tools, ques):
    llm = init_chat_model("deepseek-chat", model_provider="deepseek")
    prompt = ChatPromptTemplate.from_messages([
        ("system", "你是一个ai助手，请根据提供的上下文回答问题，确保提供所有细节，如果答案不再文中，请说'答案不在文中'。"),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}")
    ])
    tool = [tools]
    agent = create_tool_calling_agent(llm, tool, prompt)
    agent_executor = AgentExecutor(agent=agent, tools=tool, verbose=True)
    response = agent_executor.invoke({"input": ques})
    print(response)
    st.write("🤖回答：", response["output"])

def check_database_exist():
    return os.path.exists("faiss_db") and os.path.exists("faiss_db/index.faiss")

# path = "/mnt/c/Users/22695/Desktop/笔记/毕业论文/参考论文/中文/基于自适应图学习的深度多视图聚类算法研究_普敬誉.pdf"
# content = pdf_read(path)
# chunks = get_chunks(content)
# # vector_store(chunks)
# text = chunks[0]
# embedding = embedding_model.embed_documents([text])
# print(embedding)

query = "你好，请介绍一下自己。"
model = init_chat_model("deepseek-chat", model_provider="deepseek")
response = model.invoke(query)
print(response)


content='你好！我是DeepSeek，一个由深度求索公司创造的AI助手，很高兴认识你！😊\n\n让我简单介绍一下自己的特点：\n\n**基本信息：**\n- 我是纯文本AI模型，擅长理解和生成自然语言\n- 知识截止到2024年7月，拥有128K的上下文处理能力\n- 完全免费使用，没有收费计划\n\n**我能为你做什么：**\n- 回答各种问题，进行深度对话\n- 协助写作、翻译、编程等任务\n- 处理你上传的文件（图像、PDF、Word、Excel、PPT等），从中读取文字信息\n- 支持联网搜索（需要你在Web/App中手动开启）\n\n**使用方式：**\n- 可以通过网页版使用\n- 也可以通过官方应用商店下载App\n\n**我的特点：**\n- 热情细腻的交流风格\n- 乐于助人，会尽力提供详细有用的回答\n- 保持客观中立，注重安全性\n\n有什么具体问题或需要帮助的地方吗？我很乐意为你提供协助！✨' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 214, 'prompt_tokens': 10, 'total_tokens': 224, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 10}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': 'ec0d850c-3d15-4851-a9c9-813f769f902c', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b6959-77f5-77b2-8b62-e67bfcc6c405-0' usage_metadat

In [6]:
type(response)
response.content

'你好！我是DeepSeek，一个由深度求索公司创造的AI助手，很高兴认识你！😊\n\n让我简单介绍一下自己的特点：\n\n**基本信息：**\n- 我是纯文本AI模型，擅长理解和生成自然语言\n- 知识截止到2024年7月，拥有128K的上下文处理能力\n- 完全免费使用，没有收费计划\n\n**我能为你做什么：**\n- 回答各种问题，进行深度对话\n- 协助写作、翻译、编程等任务\n- 处理你上传的文件（图像、PDF、Word、Excel、PPT等），从中读取文字信息\n- 支持联网搜索（需要你在Web/App中手动开启）\n\n**使用方式：**\n- 可以通过网页版使用\n- 也可以通过官方应用商店下载App\n\n**我的特点：**\n- 热情细腻的交流风格\n- 乐于助人，会尽力提供详细有用的回答\n- 保持客观中立，注重安全性\n\n有什么具体问题或需要帮助的地方吗？我很乐意为你提供协助！✨'